In [ ]:
!nvidia-smi

Thu Apr 18 12:11:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 51.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='/content/train.json', split='train')
eval_dataset = load_dataset('json', data_files='/content/valid.json', split='train')

In [ ]:
train_dataset

Dataset({
    features: ['User Answer', 'Case', 'Feedback', 'Recommended Solution'],
    num_rows: 82
})

In [ ]:
eval_dataset

Dataset({
    features: ['User Answer', 'Case', 'Feedback', 'Recommended Solution'],
    num_rows: 10
})

In [ ]:
def create_prompt(sample):

    # Extracting features from the sample
    case = sample["Case"]
    recommended_solution = sample["Recommended Solution"]
    user_answer = sample["User Answer"]
    feedback = sample["Feedback"]

    bos_token = "<s>"
    original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    system_message = "As an interviewer, your task is to provide feedback based on the user's response, analyzing the case and recommended solution. If the user requests more information about the case, incorporate relevant details from the recommended solution into your feedback."

    # Building the prompt
    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instruction:"
    full_prompt += "\n" + system_message
    full_prompt += "\n\n### Case:"
    full_prompt += "\n" + case
    full_prompt += "\n\n### Recommended Solution:"
    full_prompt += "\n" + recommended_solution
    full_prompt += "\n\n### User Answer:"
    full_prompt += "\n" + user_answer
    full_prompt += "\n\n### Feedback:"
    full_prompt += "\n" + feedback
    full_prompt += "</s>"  # End-of-sequence token

    return full_prompt

In [ ]:
create_prompt(train_dataset[80])

"<s>### Instruction:\nAs an interviewer, your task is to provide feedback based on the user's response, analyzing the case and recommended solution. If the user requests more information about the case, incorporate relevant details from the recommended solution into your feedback.\n\n### Case:\nA successful chain of Canadian auto service stores (Autoland) has entered several markets in the United States in hopes of duplicating their success in America. The stores offer two services: 1. Retail sales of auto parts for customers who prefer to perform their own maintenance. 2. A service center for fixing any automobile problem. from an oil change to a new transmission. Since entering the U.S., Autoland has experienced $50MM in revenue with losses of $20MM. The owner is considering pulling out of the United States. You have been hired to determine if they can improve their performance or if they should exit the market.\n\n### Recommended Solution:\nAnalyze the market potential/customer segm

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [ ]:
prompt = " ### Instruction:\nAs an interviewer, your task is to provide feedback based on the user's response, analyzing the case and recommended solution. If the user requests more information about the case, incorporate relevant details from the recommended solution into your feedback.\n\n### Case:\nYour client is the National Tourist Board of the Republic of Northern Ireland (NTB). The recent peace accords have and resulting political and economic stability have helped spark a resurgence of interest in Northern Ireland, particularly in tourism and travel. The numbers and the frequency of visitors and the average duration of visits have increased. While this has led to increased commerce, travel, and tax revenues, it has also placed a strain on the country s insufficient infrastructure. The parlian ent has provided the NTB with $100 million over 3 years to promote tourism and travel by making low-interest loans and grants to private developers that match their goals and criteria. On what should it spend its money? What strategic considerations should the NTB undertake? Additional Facts (note data are disguised): Annual 1994 1996 1998 est. 1999 est. #tourists 210,000 460,000 970,000 1.7 million. Average stay 2.1 nights 3.4 nights 4.1 nights 4.5 nights $/tourist/day $45 $79 $89 $96 There is one major international airport, Belfast, which has a capacity of 3.3 million people per year. There are two major railway lines; both connect to the major cities (in Ireland and N. Ireland). There are four major highways that connect the major cities. Hotel occupancy = 59% in 1995, increasing to 79% in 1998\n\n### Recommended Solution:\nAnalyze current infrastructure use and current travel trends Determine a decision rule for each infrastructure option FACTS International airport capacity 3.3 million- currently at 97% capacity: most flights to and from London or Dublin. Most European arrivals come by train or car - extension from trips to Ireland. Highways recently re-built, but traffic congestion is vastly increasing Most Americans and Asians come by plane; few tourists arrive by boat but this number could increase with cruises. Business travelers come by plane. Largest current segment is Irish. Fastest growing tourist segments are Americans, then Italians, Germans, and Brits. Businesses would book more conferences, but there are few hotels that can accommodate their needs. Most increase in revenue comes from business travelers that bring family (demand for conferences are growing). There are two major railway lines; both connect to the major cities (in Ireland and N. Ireland). Most tourists do not stay in hotels, but stay in private bed & breakfasts; fastest growing service segment. SOLUTION ANALYSIS This question - a typical Monitor interview question tests your ability to examine and prioritize trade-offs within a well-developed framework. It's not enough to sin ply choose a strategy. Your acknowledgments must consider and weigh the facts of the case and the needs of the client. The first part of the question is asked for you -on what should the NTB focus its resources. Good answers must focus on this question, driven by the additional facts that have been provided. Clearly. the need is to ensure immediate growing pains of increased tourist traffic. Good answers should explore the infrastructure options and identify a clear strategy that accommodates both present and future needs, and should include a supporting rational. Outstanding answers explore these options and weigh the pros and cons of each, and might explore the value and return of each option (ie., an NPV analysis is always good). The second part of the question tests your creativity to look beyond the facts and determine what problems the NTB might encounter. Be careful to limit the scope of your response to those faced by the NTB. However, there is a lot of room for discussion. Good answers must address the potential for future instability: the limitations of N. Ireland as a tourist destination (ie., weather, climate, seasons), explore customer segments (ie, European vs. American vs. Asian tourists vs. Business travelers) . Outstanding answers identify specific action-oriented recommendations for these, and might also explore potential joint ventures with other members of the tourism and travel value chain.\n\n### User Answer:\nI suggest that the NTB should prioritize investing its funds in expanding the capacity of the Belfast international airport. With the airport currently operating at 97% capacity, there is a pressing need to accommodate the growing number of tourists visiting Northern Ireland. By expanding the airport's capacity, the NTB can facilitate smoother travel experiences for both domestic and international travelers, thereby enhancing the overall attractiveness of Northern Ireland as a tourist destination. Additionally, increasing the airport's capacity can support the growth of the tourism industry by attracting more visitors and stimulating economic activity in the region. To calculate the potential return on investment, we can conduct a cost-benefit analysis considering factors such as the projected increase in tourist arrivals, the revenue generated from airport usage fees, and the impact on local businesses and employment opportunities.\n\n### Feedback: "

In [ ]:
generate_response(prompt, model)

'<s> \nYour suggestion to expand the capacity of the Belfast international airport is certainly a valid one, given the increasing number of tourists visiting Northern Ireland and the current capacity constraints of the airport. However, it is important to note that while expanding the airport may help accommodate the growing demand, it may also lead to increased costs, including aircraft lease, maintenance, staffing, and other operational expenses. Moreover, an expansion would require significant time and resources for planning, construction, and financing the project. \n\nOn the other hand, your proposed cost-benefit analysis is a comprehensive approach to evaluating the potential return on investment from the expansion project. By considering factors such as the projected increase in tourist arrivals, the revenue generated from airport usage fees, and the impact on local businesses and employment opportunities, you are providing a more informed perspective on the potential benefits o

In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/mistral/mistral_instruct_generation",
    max_steps=100,
    per_device_train_batch_size=2,
    warmup_steps=0.03,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    lr_scheduler_type='constant',
    do_train=True,
    do_eval=True,
    overwrite_output_dir=True,
    logging_dir="/content/mistral/logs",
    save_total_limit=2,
    gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
)

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    max_seq_length=1024,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=create_prompt,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.074400
20,1.867300
30,1.714200
40,1.595100
50,1.452100
60,1.326300
70,1.161800
80,1.003700
90,0.852200
100,0.721000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=100, training_loss=1.3768110704421996, metrics={'train_runtime': 16351.9176, 'train_samples_per_second': 0.049, 'train_steps_per_second': 0.006, 'total_flos': 3.468962088930509e+16, 'train_loss': 1.3768110704421996, 'epoch': 9.76})

In [ ]:
trainer.save_model("/content/mistral/mistral_instruct_generation")

# Inference

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/mistral/mistral_instruct_generation")

In [8]:
eval_prompt = " ### Instruction:\nAs an interviewer, your task is to provide feedback based on the user's response, analyzing the case and recommended solution. If the user requests more information about the case, incorporate relevant details from the recommended solution into your feedback.\n\n### Case:\nA successful chain of Canadian auto service stores (Autoland) has entered several markets in the United States in hopes of duplicating their success in America. The stores offer two services: 1. Retail sales of auto parts for customers who prefer to perform their own maintenance. 2. A service center for fixing any automobile problem. from an oil change to a new transmission. Since entering the U.S., Autoland has experienced $50MM in revenue with losses of $20MM. The owner is considering pulling out of the United States. You have been hired to determine if they can improve their performance or if they should exit the market.\n\n### Recommended Solution:\nAnalyze the market potential/customer segments Competitive Situation: What is the competitive situation in Canada? A: We are the major player (few local stores) Are we providing the same services in Canada as in the US? A: Yes Do we have strong competition in the U.S.? A: Yes, a national chain of stores in the exact format as Autoland exists in the U.S. They basically copied our Canadian format and have about 10 locations in every major city. They are very profitable in all cities including our U.S. markets. Assumptions: Due to size. I would guess they have superior buying power over Autoland in the U.S. Is this true? A: No, we have the same cost structure due to our presence in Canada. Assumption: The market has potential due to the con petitor's perfom ance. Key is to determine why they are out- performing Autoland. Autoland Capabilities: Assumption: We actually have to businesses under one roof, is one more profitable than the other? A: In Canada - no. But in the U.S. we are profitable in retail sales and losing heavily on the service center. Are the costs associated with each side of the business different? A: Yes, the service center is much more expensive to operate, we have to pay mechanics and have high fixed costs. Assumption: We are profitable in retail, but losing in service. We attract the wrong consumer. Market/Customers: Autoland provides two services, are the customers for each service different? A: Yes. The customers that shop for retail parts typically have lower to middle incomes and are trying to save a few dollars by performing their own maintenance. The customers who utilize the service center have higher incomes and no interest in fixing their own car. Assumption: We are attempting to attract two distinct customer segments. Are we doing this successfully? A: We are not sure, how would you help us determine if we are? Factors: Marketing: We do the same as competition. Pricing. Identical to competition. Location. Different, we are located in the inner cities to save money on leases. Where is the competition located? A: Between the inner city and the suburbs (on the border) Assumptions / Recommendations: Our location is great for the retail sales business, but prohibits heavy use of the service center due to the distribution of income between the inner city vs. suburbs. In new markets, locate between the lower and upper income areas to attract both segments. In existing markets, move, or drop the service business and retain the profitable retail portion.\n\n### User Answer:\nTo assess Autoland's performance in the U.S. market and determine the best course of action, we need to analyze various factors:\n\nMarket potential and customer segments: We need to understand the demand for auto parts retail sales and service center offerings in different regions.\nCompetitive situation: Identifying the competitive landscape in both Canada and the U.S. is crucial to understand Autoland's position and performance relative to competitors.\nAutoland's capabilities: Analyzing the profitability of each segment of the business (retail sales vs. service center) will help identify strengths and weaknesses.\nCustomer demographics: Understanding the differences in customer segments for retail sales and service center operations is essential for targeted marketing and service delivery.\nMarketing, pricing, and location: Assessing Autoland's marketing strategies, pricing policies, and location choices compared to competitors will provide insights into areas for improvement.\nBased on the information provided, it seems that Autoland's service center is struggling due to high operating costs and possibly mismatched customer demographics. Therefore, a strategic reevaluation of location and target market alignment is necessary. Moving or adjusting the service center's focus while retaining the profitable retail sales business may be a viable option. Additionally, expanding into new markets with a more balanced location strategy could help attract both customer segments.\n\n### Feedback: "

In [10]:
eval_prompt

" ### Instruction:\nAs an interviewer, your task is to provide feedback based on the user's response, analyzing the case and recommended solution. If the user requests more information about the case, incorporate relevant details from the recommended solution into your feedback.\n\n### Case:\nA successful chain of Canadian auto service stores (Autoland) has entered several markets in the United States in hopes of duplicating their success in America. The stores offer two services: 1. Retail sales of auto parts for customers who prefer to perform their own maintenance. 2. A service center for fixing any automobile problem. from an oil change to a new transmission. Since entering the U.S., Autoland has experienced $50MM in revenue with losses of $20MM. The owner is considering pulling out of the United States. You have been hired to determine if they can improve their performance or if they should exit the market.\n\n### Recommended Solution:\nAnalyze the market potential/customer segmen

In [9]:
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=1000, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 ### Instruction:
As an interviewer, your task is to provide feedback based on the user's response, analyzing the case and recommended solution. If the user requests more information about the case, incorporate relevant details from the recommended solution into your feedback.

### Case:
A successful chain of Canadian auto service stores (Autoland) has entered several markets in the United States in hopes of duplicating their success in America. The stores offer two services: 1. Retail sales of auto parts for customers who prefer to perform their own maintenance. 2. A service center for fixing any automobile problem. from an oil change to a new transmission. Since entering the U.S., Autoland has experienced $50MM in revenue with losses of $20MM. The owner is considering pulling out of the United States. You have been hired to determine if they can improve their performance or if they should exit the market.

### Recommended Solution:
Analyze the market potential/customer segments Compe